In [ ]:
import requests
import os
from calendar import monthrange
from multiprocessing import Pool


downld_sea = "Storm and sea level/data/sea/"
downld_wind = "Storm and sea level/data/wind/"
#дата хранится в виде массива формата [y, m, d], start < end
start = [2011, 4, 1]
end = [2019, 1, 1]

# переменная для регулирования количества параллельных процессов
q_amnt = 12

In [ ]:
# вспомогающая функция для увеличения даты на заданное количество дней 
def add_days(date, days):
    dsamnt = monthrange(date[0], date[1])
    date[2] += days
    if date[2] > dsamnt[1]:
        date[2] -= dsamnt[1]
        date[1] += 1
    if date[1] == 13:
        date[1] = 1
        date[0] += 1

Скачиваем файлы с данными об уровне моря

In [ ]:
# функция для скачивания одного файла
def fetch_url_sea(item):
    cur = item[0]
    cursave = item[1]
    name = item[2]
    login = item[3]
    pwd = item[4]
    with open(cursave, "wb") as f:
        response = requests.get(cur, auth=(login, pwd), stream=True)
        print("Downloading %s Status code %s" % (name, response.status_code))
        for chunk in response:
            f.write(chunk)
    print("%s is Done!" % name)


cdate = list() # текущая дата
cdate.append(start[0])
cdate.append(start[1])
cdate.append(start[2])
basename = "ssh_grids_v1812_"
endname = "12.nc"
url = "https://podaac-tools.jpl.nasa.gov/drive/files/allData/merged_alt/L4/cdr_grid/"
savefile = downld_sea
p = Pool(q_amnt) # создаем пул для распараллелирования процессов
queries = list()
while cdate[0] != end[0] or cdate[1] != end[1]:
    if len(queries) == q_amnt:
        p.imap(fetch_url_sea, queries)
        queries = list()
    name = str(cdate[0])
    if len(str(cdate[1])) == 1:
        name += '0'
    name += str(cdate[1])
    if len(str(cdate[2])) == 1:
        name += '0'
    name += str(cdate[2])
    name = basename + name + endname
    cur = url + name
    cursave = savefile + name 
    login = # введите свой логин здесь, например "user"
    pwd = # введите свой пароль здесь
    response = requests.get(cur, auth=(login, pwd), stream=True)
    if response.status_code != 200: # файла с таким названием нет
        print("Downloading %s Status code ERROR %s" % (name, response.status_code))
        add_days(cdate, 1)
    else:
        #добавляем параметры [адрес, место сохранения, имя файла, логин, пароль] в очередь на запуск функции
        itm = [cur, cursave, name, login, pwd] 
        queries.append(itm)
        add_days(cdate, 5)
        
    
p.imap(fetch_url_sea, queries)  
p.close()
p.join()
print("Download finished!")

Скачиваем файлы с данными о скорости ветра. Функции построены так же, только для скачивания данных по ветру не требуется логин и пароль.

In [ ]:
# функция для скачивания одного файла
def fetch_url_wind(item):
    cur = item[0]
    cursave = item[1]
    name = item[2]
    with open(cursave, "wb") as f:
        response = requests.get(cur, stream=True)
        if response.status_code != 200:
            print("Downloading %s Status code ERROR %s" % (name, response.status_code))
        else:
            print("Downloading %s Status code %s" % (name, response.status_code))
            for chunk in response:
                f.write(chunk)
    print("%s is Done!" % name)

cdate = list()
cdate.append(start[0])
cdate.append(start[1])
cdate.append(start[2])
basename = "wnd10m.l.gdas."
endname = ".grb2"
url = "https://nomads.ncdc.noaa.gov/data/cfsr/"
savefile = downld_wind
p = Pool(q_amnt)
queries = list()
while cdate[0] != end[0] or cdate[1] != end[1]:
    if cdate[0] > 2011 or (cdate[0] == 2011 and cdate[1] > 3):
        # начиная с этого момента файлы нового формата находятся в другом месте
        url = "https://nomads.ncdc.noaa.gov/modeldata/cfsv2_analysis_timeseries/"
        endname = ".grib2"
    name = str(cdate[0])
    if len(str(cdate[1])) == 1:
        name += '0'
    name += str(cdate[1])
    cur = url + str(cdate[0])+"/"+name +"/" + basename + name + endname
    cursave = savefile + basename + name + endname
    itm = [cur, cursave, basename + name + endname]
    queries.append(itm)
    cdate[1] += 1
    if cdate[1] == 13:
        cdate[1] = 1
        cdate[0] += 1
        
p.imap(fetch_url_wind, queries)    
p.close()
p.join()
print("Download finished!")